In [5]:
class Factory(ManufacturingProcess):
    # 기본값. 2행 5열. Product 두개로 시작.
    row = 2
    col = 5
    numStartProduct = 2

NameError: name 'ManufacturingProcess' is not defined

# def \__init__( )

In [ ]:
    def __init__(self, strFileName, breakProb):    # breakProb이 뭔지 모르겠다. 특정 station을 탈출할 확률?
                                                   # main.py에 0.3인걸 보니 30% 확률로 탈출인 듯.
        self.breakProb = breakProb    # main.py에서는 0.3
        self.waitingProduct = ProductionList(strFileName)    # 생산 라인에 투입되기 전 product들.
        self.completedProduct = ProductionList('')    # 공정을 마친 product 넣기 위한 빈 리스트.

        self.processes = [[[] for j in range(self.col)] for i in range(self.row)]    # 초기 변수로 받은 행렬값으로 생산공정 생성.
        for i in range(self.row):    # 2
            for j in range(self.col):    # 5
                if i == 0:
                    self.processes[i][j] = ManufacturingProcess('queue')    # 첫 번째 라인은 Queue 방식으로.
                if i == 1:
                    self.processes[i][j] = ManufacturingProcess('stack')    # 두 번째 라인은 Stack 방식으로.

# def selectLine( )

In [ ]:
    def selectLine(self, matCost):    # Return line = 1 or 2 by selecting a line by dynamic
                                      # programming. matCost = (Line) X (Processor)
                                      # Line = this.row, Processor = this.col
                                      # 아마 이 메소드는 process 모두 끝나고 어느 라인을 선택할지 사용하는 메소드인듯
                                      # 아니고서야 이런 미친짓 할리 없다.
        matShortestCost = np.zeros((self.row, self.col))    # memoization table.
        matPrevLine = np.zeros((self.row, self.col))    # retrace table

        for i in range(self.row):
            matShortestCost[i][0] = matCost[i][0]    # memoization의 첫 번째 value는 matCost의 첫 번째 value.

        for i in range(1, self.col):    # 1, 2, 3, 4
            for j in range(self.row):    # 0, 1
                if matShortestCost[0][i-1] + matCost[j][1] < matShortestCost[1][j-1] + matCost[j][i] :
                    matShortestCost[j][i] = matShortestCost[0][i-1] + matCost[j][i]
                    matPrevLine[j][i] = int(0)
                else:
                    matShortestCost[j][i] = matShortestCost[1][i-1] + matCost[j][i]
                    matPrevLine[j][i] = int(1)

        if matShortestCost[0][self.col-1] < matShortestCost[1][self.col-1] :
            endLine = int(0)
        else:
            endLine = int(1)

        for i in range(self.col-1, 0, -1):    # 4, 3, 2, 1
            endLine = int(matPrevLine[endLine][i])
            
        line = endLine
        
        print('======================')
        print('Cost Matrix : ')
        print(matShortestCost)
        print('Retrace Matrix : ')
        print(matPrevLine)
        return line

# def getCostMatrix( )
- cost = 특정 station에 jam되있는 product의 개수?
- cost = time to process to the next station?

In [ ]:
    def getCostMatrix(self):    # 위의 selectLine() 메소드에서 활용할 matCost 변수를 계산하는 메소드.
        matCost = np.zeros((self.row, self.col))    # 똑같이 초기값은 2x5 영행렬.
        for i in range(self.row):    # 2
            for j in range(self.col):    # 5
                matCost[i][j] = self.processes[i][j].getSize()    # 생산 공정 속 특정 stiation의 size. 
                                                                  # station에 몇 개의 product가 있는지.
                                                                  # j=0이면 Queue, j=1이면 Stack.
        return matCost